In [17]:
import wandb
import pandas as pd

api = wandb.Api()

DUMMY_RUN = "rap1ide/slice_inflate/xx190fww"

pd.set_option('display.max_rows', 48)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

def get_agg_dict(filtered_frame):
    agg_dict = {
        n: 'first' if t != float else 'mean' for n,t in zip(filtered_frame.columns,filtered_frame.dtypes)
    }
    return agg_dict

# get n chunks of a list
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def join_cols(frame, first_col_key, second_col_key, join_str="+", drop_second=True):
    first_col = frame[first_col_key].astype(str)
    second_col = frame[second_col_key].astype(str)
    frame[first_col_key] = first_col + join_str + second_col
    if drop_second:
        frame.drop(second_col_key, axis=1, inplace=True)
    else:
        frame.drop(first_col_key, axis=1, inplace=True)
    return frame

In [2]:
settings = {
   "0:1": dict(description="Clinical standard",
      first_view="p2CH",
      second_view="p4CH",
      prescan_res ="$(1.5mm)^3$",
      prescan_type ="GT",
      slice_res ="$(1.5mm)^2$",
      slice_type="GT",
   ),
   "1:2": dict(description="Clinical standard",
      first_view="2CH",
      second_view="4CH",
      prescan_res ="$(1.5mm)^3$",
      prescan_type ="GT",
      slice_res ="$(1.5mm)^2$",
      slice_type="GT",
   ),
   "2:3": dict(description="Clinical standard",
      first_view="2CH",
      second_view="SA",
      prescan_res ="$(1.5mm)^3$",
      prescan_type ="GT",
      slice_res ="$(1.5mm)^2$",
      slice_type="GT",
   ),
   "3:9": dict(description="Mean out of 6 Random",
      first_view="RND",
      second_view="RND",
      prescan_res ="$(1.5mm)^3$",
      prescan_type ="GT",
      slice_res ="$(1.5mm)^2$",
      slice_type="GT",
   ),
   "9:10": dict(description="Optimized",
      first_view="OPT",
      second_view="OPT",
      prescan_res ="$(1.5mm)^3$",
      prescan_type ="GT",
      slice_res ="$(1.5mm)^2$",
      slice_type="GT",
   ),

   "10:11": dict(description="Clinical standard",
      first_view="2CH",
      second_view="4CH",
      prescan_res ="$(6mm)^3$",
      prescan_type ="GT",
      slice_res ="$(1.5mm)^2$",
      slice_type="GT",
   ),
   "11:12": dict(description="Optimized",
      first_view="OPT",
      second_view="OPT",
      prescan_res ="$(6mm)^3$",
      prescan_type ="GT",
      slice_res ="$(1.5mm)^2$",
      slice_type="GT",
   ),

   "12:13": dict(description="Clinical standard",
      first_view="2CH",
      second_view="4CH",
      prescan_res ="$(6mm)^3$",
      prescan_type ="SEG",
      slice_res ="$(1.5mm)^2$",
      slice_type="SEG",
   ),
   "13:14": dict(description="Optimized",
      first_view="OPT",
      second_view="OPT",
      prescan_res ="$(6mm)^3$",
      prescan_type ="SEG",
      slice_res ="$(1.5mm)^2$",
      slice_type="SEG",
   ),

   "14:15": dict(description="Clinical standard",
      first_view="2CH",
      second_view="4CH",
      prescan_res ="$(6mm)^3$",
      prescan_type ="SEG",
      slice_res ="$(6mm)^3$",
      slice_type="SEG",
   ),
   "15:16": dict(description="Optimized",
      first_view="OPT",
      second_view="OPT",
      prescan_res ="$(6mm)^3$",
      prescan_type ="SEG",
      slice_res ="$(6mm)^3$",
      slice_type="SEG",
   ),
}

# Define run paths of wandb MMWHS

In [3]:
mmwhs_run_path_list = list(reversed([
'rap1ide/slice_inflate/runs/zk9i0rrx',
'rap1ide/slice_inflate/runs/ge0mauw4',
'rap1ide/slice_inflate/runs/t1dl1x69',
'rap1ide/slice_inflate/runs/fjmhxre7',
'rap1ide/slice_inflate/runs/vm0sjun1',
'rap1ide/slice_inflate/runs/p2eoy8z0',
'rap1ide/slice_inflate/runs/9gtmnnfv',
'rap1ide/slice_inflate/runs/hvvddqer',
'rap1ide/slice_inflate/runs/kfzdcepx',
'rap1ide/slice_inflate/runs/5m1et0aq',
'rap1ide/slice_inflate/runs/c4gqhk82',
'rap1ide/slice_inflate/runs/okxvp25d',
'rap1ide/slice_inflate/runs/5jj2gdhk',
'rap1ide/slice_inflate/runs/ez1e3vr3',
'rap1ide/slice_inflate/runs/upt7xjkn',
'rap1ide/slice_inflate/runs/vpce07jm',
'rap1ide/slice_inflate/runs/qmvin2ux',
'rap1ide/slice_inflate/runs/wl068f1l',
'rap1ide/slice_inflate/runs/dkgq89df',
'rap1ide/slice_inflate/runs/4vrszjn4',
'rap1ide/slice_inflate/runs/jiolfg7j',
'rap1ide/slice_inflate/runs/5estavq0',
'rap1ide/slice_inflate/runs/eoc7me0e',
'rap1ide/slice_inflate/runs/8ac6id01',
'rap1ide/slice_inflate/runs/wao2b16c',
'rap1ide/slice_inflate/runs/sih8wc65',
'rap1ide/slice_inflate/runs/pmyls33r',
'rap1ide/slice_inflate/runs/u8bdwaji',
'rap1ide/slice_inflate/runs/lad69u2m',
'rap1ide/slice_inflate/runs/jsw6an38',
'rap1ide/slice_inflate/runs/x5w8hml2',
'rap1ide/slice_inflate/runs/gobs9276',
    'rap1ide/slice_inflate/runs/w0bgwxn3',
    'rap1ide/slice_inflate/runs/6plcdxdr',
    'rap1ide/slice_inflate/runs/e975wi0p',
    'rap1ide/slice_inflate/runs/ukoauy9x',
    'rap1ide/slice_inflate/runs/gd12cq5u',
    'rap1ide/slice_inflate/runs/cjbdy78r',
    'rap1ide/slice_inflate/runs/b8rh2d09',
    'rap1ide/slice_inflate/runs/9xe5ql4h',
    'rap1ide/slice_inflate/runs/cte64bvv',
    'rap1ide/slice_inflate/runs/8vbmnmre',
    'rap1ide/slice_inflate/runs/4te8z4rk',
    'rap1ide/slice_inflate/runs/ay7bgyd5',
    'rap1ide/slice_inflate/runs/sfh7xw2p',
    'rap1ide/slice_inflate/runs/zdpeonni',
    'rap1ide/slice_inflate/runs/j16qb5rm',
    'rap1ide/slice_inflate/runs/4kdmc0ch',
    'rap1ide/slice_inflate/runs/mag0njn8',
    'rap1ide/slice_inflate/runs/6d19um8q',
    'rap1ide/slice_inflate/runs/g5hai89a',
    'rap1ide/slice_inflate/runs/31zvi2a0',
    'rap1ide/slice_inflate/runs/f0mu5sbk',
    'rap1ide/slice_inflate/runs/krljgne0',
    'rap1ide/slice_inflate/runs/n41shvul',
    'rap1ide/slice_inflate/runs/wzmr06qe',
    'rap1ide/slice_inflate/runs/0spvd8jj',
    'rap1ide/slice_inflate/runs/jl7prqo8',
    'rap1ide/slice_inflate/runs/n2rf1ae4',
    'rap1ide/slice_inflate/runs/5im33z1m',
    'rap1ide/slice_inflate/runs/j7h289df',
    'rap1ide/slice_inflate/runs/g85qf8ss',
    'rap1ide/slice_inflate/runs/q3963sr5',
    'rap1ide/slice_inflate/runs/tww9c2fm',
    'rap1ide/slice_inflate/runs/y9m2v1b3',
    'rap1ide/slice_inflate/runs/0snhugvl',
    'rap1ide/slice_inflate/runs/a2go54je',
    'rap1ide/slice_inflate/runs/81zajv9e',
]))

In [4]:
mmwhs_run_name_list = list(reversed([
'20240126__03_45_52_progressive-granite_stage-2_fold-1',
'20240126__06_34_54_mad-prefix_stage-2_fold-0',
'20240126__03_45_52_progressive-granite_stage-1_fold-1',
'20240126__06_34_54_mad-prefix_stage-1_fold-0',
'20240126__03_45_52_crazy-core_stage-4_fold-0',
'20240125__21_15_50_leather-plot_stage-4_fold-2',
'20240126__03_45_52_crazy-core_stage-3_fold-0',
'20240125__21_15_50_leather-plot_stage-3_fold-2',
'20240126__03_45_52_crazy-core_stage-2_fold-0',
'20240125__21_15_50_leather-plot_stage-2_fold-2',
'20240126__03_45_52_crazy-core_stage-1_fold-0',
'20240125__21_15_50_leather-plot_stage-1_fold-2',
'20240125__21_11_31_agile-directive_stage-4_fold-2',
'20240125__21_11_31_agile-directive_stage-3_fold-2',
'20240125__21_15_50_magenta-glass_stage-4_fold-1',
'20240125__21_11_31_agile-directive_stage-2_fold-2',
'20240125__21_15_50_magenta-glass_stage-3_fold-1',
'20240125__21_11_31_agile-directive_stage-1_fold-2',
'20240125__21_15_50_magenta-glass_stage-2_fold-1',
'20240125__21_11_31_coal-adjugate_stage-4_fold-1',
'20240125__21_15_50_magenta-glass_stage-1_fold-1',
'20240125__21_11_31_coal-adjugate_stage-3_fold-1',
'20240125__21_15_50_plain-mask_stage-4_fold-0',
'20240125__21_11_31_coal-adjugate_stage-2_fold-1',
'20240125__21_11_31_coal-adjugate_stage-1_fold-1',
'20240125__21_15_50_plain-mask_stage-3_fold-0',
'20240125__21_11_31_sticky-league_stage-4_fold-0',
'20240125__21_11_31_sticky-league_stage-3_fold-0',
'20240125__21_15_50_plain-mask_stage-2_fold-0',
'20240125__21_11_31_sticky-league_stage-2_fold-0',
'20240125__21_15_50_plain-mask_stage-1_fold-0',
'20240125__21_11_31_sticky-league_stage-1_fold-0',
    '20240125__17_37_07_active-caramel_stage-1_fold-2',
    '20240125__17_37_07_wide-cube_stage-1_fold-1',
    '20240125__17_37_07_lazy-object_stage-1_fold-0',
    '20240125__15_12_13_plain-pizza_stage-1_fold-2',
    '20240125__15_12_13_chartreuse-limestone_stage-1_fold-1',
    '20240125__15_12_13_human-bracket_stage-1_fold-0',
    '20240125__14_28_50_raw-gravity_stage-1_fold-2',
    '20240125__14_28_50_clever-lion_stage-1_fold-1',
    '20240125__14_28_50_cyan-negation_stage-1_fold-0',
    '20240125__11_31_39_chill-keyframe_stage-1_fold-2',
    '20240125__11_31_39_vibrant-frequency_stage-1_fold-1',
    '20240125__11_31_39_warm-lepton_stage-1_fold-0',
    '20240125__11_03_37_savory-hangar_stage-1_fold-2',
    '20240125__11_03_37_ascent-incubator_stage-1_fold-1',
    '20240125__11_03_37_crunchy-pouf_stage-1_fold-0',
    '20240125__10_35_37_uniform-cycle_stage-1_fold-2',
    '20240125__10_35_37_mean-goal_stage-1_fold-1',
    '20240125__10_35_37_modern-riser_stage-1_fold-0',
    '20240125__10_07_01_nullary-supervisor_stage-1_fold-2',
    '20240125__10_07_01_tangy-bow_stage-1_fold-1',
    '20240125__10_07_01_cold-yeast_stage-1_fold-0',
    '20240125__09_38_16_clever-relish_stage-1_fold-2',
    '20240125__09_38_16_brute-ion_stage-1_fold-1',
    '20240125__09_38_16_rectilinear-level_stage-1_fold-0',
    '20240125__09_10_32_free-inflection_stage-1_fold-2',
    '20240125__09_10_32_wide-crocodile_stage-1_fold-1',
    '20240125__09_10_32_creamy-web_stage-1_fold-0',
    '20240125__08_41_32_celeste-dataframe_stage-1_fold-2',
    '20240125__08_41_32_deafening-filler_stage-1_fold-1',
    '20240125__08_41_32_brownian-chronology_stage-1_fold-0',
    '20240125__08_14_19_furious-asset_stage-1_fold-2',
    '20240125__08_14_19_deterministic-convergence_stage-1_fold-1',
    '20240125__08_14_19_free-editor_stage-1_fold-0',
    '20240125__07_46_26_violent-relativity_stage-1_fold-2',
    '20240125__07_46_26_free-graph_stage-1_fold-1',
    '20240125__07_46_26_trusting-rosin_stage-1_fold-0',
]))

import numpy as np
new_mmwhs_sorted_args = np.argsort(mmwhs_run_name_list)#[36:64]

mmwhs_run_name_list = list(np.array(mmwhs_run_name_list)[new_mmwhs_sorted_args])
mmwhs_run_path_list = list(np.array(mmwhs_run_path_list)[new_mmwhs_sorted_args])

In [5]:
mmwhs_run_name_list

['20240125__07_46_26_free-graph_stage-1_fold-1',
 '20240125__07_46_26_trusting-rosin_stage-1_fold-0',
 '20240125__07_46_26_violent-relativity_stage-1_fold-2',
 '20240125__08_14_19_deterministic-convergence_stage-1_fold-1',
 '20240125__08_14_19_free-editor_stage-1_fold-0',
 '20240125__08_14_19_furious-asset_stage-1_fold-2',
 '20240125__08_41_32_brownian-chronology_stage-1_fold-0',
 '20240125__08_41_32_celeste-dataframe_stage-1_fold-2',
 '20240125__08_41_32_deafening-filler_stage-1_fold-1',
 '20240125__09_10_32_creamy-web_stage-1_fold-0',
 '20240125__09_10_32_free-inflection_stage-1_fold-2',
 '20240125__09_10_32_wide-crocodile_stage-1_fold-1',
 '20240125__09_38_16_brute-ion_stage-1_fold-1',
 '20240125__09_38_16_clever-relish_stage-1_fold-2',
 '20240125__09_38_16_rectilinear-level_stage-1_fold-0',
 '20240125__10_07_01_cold-yeast_stage-1_fold-0',
 '20240125__10_07_01_nullary-supervisor_stage-1_fold-2',
 '20240125__10_07_01_tangy-bow_stage-1_fold-1',
 '20240125__10_35_37_mean-goal_stage-1_f

### Print run lists for convenience MMWHS

In [6]:
for c_id, (nc, pc) in enumerate(zip(chunks(mmwhs_run_name_list, 4), chunks(mmwhs_run_path_list, 4))):
    print(f"## {c_id}")
    for n, p in zip(nc, pc):
        print(n, "\t", p)
print()


## 0
20240125__07_46_26_free-graph_stage-1_fold-1 	 rap1ide/slice_inflate/runs/a2go54je
20240125__07_46_26_trusting-rosin_stage-1_fold-0 	 rap1ide/slice_inflate/runs/81zajv9e
20240125__07_46_26_violent-relativity_stage-1_fold-2 	 rap1ide/slice_inflate/runs/0snhugvl
20240125__08_14_19_deterministic-convergence_stage-1_fold-1 	 rap1ide/slice_inflate/runs/tww9c2fm
## 1
20240125__08_14_19_free-editor_stage-1_fold-0 	 rap1ide/slice_inflate/runs/y9m2v1b3
20240125__08_14_19_furious-asset_stage-1_fold-2 	 rap1ide/slice_inflate/runs/q3963sr5
20240125__08_41_32_brownian-chronology_stage-1_fold-0 	 rap1ide/slice_inflate/runs/g85qf8ss
20240125__08_41_32_celeste-dataframe_stage-1_fold-2 	 rap1ide/slice_inflate/runs/5im33z1m
## 2
20240125__08_41_32_deafening-filler_stage-1_fold-1 	 rap1ide/slice_inflate/runs/j7h289df
20240125__09_10_32_creamy-web_stage-1_fold-0 	 rap1ide/slice_inflate/runs/n2rf1ae4
20240125__09_10_32_free-inflection_stage-1_fold-2 	 rap1ide/slice_inflate/runs/0spvd8jj
20240125__09_1

In [43]:
mmwhs_run_dict = {
   ######## 00 ########
   "20240125__07_46_26_trusting-rosin_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/81zajv9e",
      id=0,
      fold=0,
   ),
   "20240125__07_46_26_free-graph_stage-1_fold-1": dict(
      access_key="rap1ide/slice_inflate/a2go54je",
      id=0,
      fold=1,
   ),
   "20240125__07_46_26_violent-relativity_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/0snhugvl",
      id=0,
      fold=2,
   ),

   ######## 01 ########
   # "20240125__08_14_19_free-editor_stage-1_fold-0": dict(
   #    access_key="rap1ide/slice_inflate/y9m2v1b3",
   #    id=1,
   #    fold=0,
   # ),
   "20240130__22_22_28_approximate-hop_fold-0_stage-4": dict( # TODO new
      access_key="rap1ide/slice_inflate/fgfu6jen",
      id=1,
      fold=0,
   ),
   "20240125__08_14_19_deterministic-convergence_stage-1": dict(
      access_key="rap1ide/slice_inflate/tww9c2fm",
      id=1,
      fold=1,
   ),
   "20240125__08_14_19_furious-asset_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/q3963sr5",
      id=1,
      fold=2,
   ),

   ######## 02 ########
   "20240125__08_41_32_brownian-chronology_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/g85qf8ss",
      id=2,
      fold=0,
   ),
   "20240125__08_41_32_deafening-filler_stage-1_fold-1": dict(
      access_key="rap1ide/slice_inflate/j7h289df",
      id=2,
      fold=1,
   ),
   "20240125__08_41_32_celeste-dataframe_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/5im33z1m",
      id=2,
      fold=2,
   ),

   ######## 03 ########
   "20240125__09_10_32_creamy-web_stage-1_fold-0 ": dict(
      access_key="rap1ide/slice_inflate/n2rf1ae4",
      id=3,
      fold=0,
   ),
   "20240125__09_10_32_wide-crocodile_stage-1_fold-1": dict(
      access_key="rap1ide/slice_inflate/jl7prqo8",
      id=3,
      fold=1,
   ),
   "20240125__09_10_32_free-inflection_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/0spvd8jj",
      id=3,
      fold=2,
   ),

   ######## 04 ########
   "20240125__09_38_16_rectilinear-level_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/wzmr06qe",
      id=4,
      fold=0,
   ),
   "20240125__09_38_16_brute-ion_stage-1_fold-1": dict(
      access_key="rap1ide/slice_inflate/n41shvul",
      id=4,
      fold=1,
   ),
   "20240125__09_38_16_clever-relish_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/krljgne0",
      id=4,
      fold=2,
   ),

   ######## 05 ########
   "20240125__10_07_01_cold-yeast_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/f0mu5sbk",
      id=5,
      fold=0,
   ),
   "20240125__10_07_01_tangy-bow_stage-1_fold-1": dict(
      access_key="rap1ide/slice_inflate/31zvi2a0",
      id=5,
      fold=1,
   ),
   "20240125__10_07_01_nullary-supervisor_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/g5hai89a",
      id=5,
      fold=2,
   ),

   ######## 06 ########
   "20240125__10_35_37_modern-riser_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/6d19um8q",
      id=6,
      fold=0,
   ),
   "20240125__10_35_37_mean-goal_stage-1_fold-1": dict(
      access_key="rap1ide/slice_inflate/mag0njn8",
      id=6,
      fold=1,
   ),
   "20240125__10_35_37_uniform-cycle_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/4kdmc0ch",
      id=6,
      fold=2,
   ),

   ######## 07 ########
   "20240125__11_03_37_crunchy-pouf_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/j16qb5rm",
      id=7,
      fold=0,
   ),
   "20240125__11_03_37_savory-hangar_stage-1_fold-2 ": dict(
      access_key="rap1ide/slice_inflate/zdpeonni",
      id=7,
      fold=1,
   ),
   "20240125__11_03_37_savory-hangar_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/sfh7xw2p",
      id=7,
      fold=2,
   ),
   ######## 08 ########
   "20240125__11_31_39_warm-lepton_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/ay7bgyd5",
      id=8,
      fold=0,
   ),
   "20240125__11_31_39_vibrant-frequency_stage-1_fold-1": dict(
      access_key="rap1ide/slice_inflate/4te8z4rk",
      id=8,
      fold=1,
   ),
   "20240125__11_31_39_chill-keyframe_stage-1_fold-2 ": dict(
      access_key="rap1ide/slice_inflate/8vbmnmre",
      id=8,
      fold=2,
   ),

   ######## 09 ########
   "20240130__22_22_28_approximate-hop_fold-0_stage-3": dict( # TODO new!
      access_key="rap1ide/slice_inflate/jz89jezu",
      id=9,
      fold=0,
   ),
   "dummy28": dict(
      access_key=DUMMY_RUN,
      id=9,
      fold=1,
   ),
   "dummy29": dict(
      access_key=DUMMY_RUN,
      id=9,
      fold=2,
   ),

   ######## 10 ########
   "20240125__14_28_50_cyan-negation_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/cte64bvv",
      id=10,
      fold=0,
   ),
   "20240125__14_28_50_clever-lion_stage-1_fold-1": dict(
      access_key="rap1ide/slice_inflate/runs/9xe5ql4h",
      id=10,
      fold=1,
   ),
   "20240125__14_28_50_raw-gravity_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/runs/b8rh2d09",
      id=10,
      fold=2,
   ),

   ######## 11 ########
   "20240125__21_11_31_sticky-league_stage-3_fold-0 ": dict(
      access_key="rap1ide/slice_inflate/runs/u8bdwaji",
      id=11,
      fold=0,
   ),
   "20240125__21_11_31_coal-adjugate_stage-3_fold-1": dict(
      access_key="rap1ide/slice_inflate/runs/5estavq0",
      id=11,
      fold=1,
   ),
   "20240125__21_11_31_agile-directive_stage-3_fold-2": dict(
      access_key="rap1ide/slice_inflate/runs/ez1e3vr3",
      id=11,
      fold=2,
   ),
   ######## 12 ########
   "20240125__15_12_13_human-bracket_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/cjbdy78r",
      id=12,
      fold=0,
   ),
   "20240125__15_12_13_chartreuse-limestone_stage-1_fold-1": dict(
      access_key=" rap1ide/slice_inflate/runs/gd12cq5u",
      id=12,
      fold=1,
   ),
   "20240125__15_12_13_plain-pizza_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/runs/ukoauy9x",
      id=12,
      fold=2,
   ),

   ######## 13 ########
   "20240125__21_15_50_plain-mask_stage-3_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/sih8wc65",
      id=13,
      fold=0,
   ),
   "20240125__21_15_50_magenta-glass_stage-3_fold-1": dict(
       access_key="rap1ide/slice_inflate/runs/qmvin2ux",
      id=13,
      fold=1,
   ),
   "20240125__21_15_50_leather-plot_stage-3_fold-2": dict(
      access_key="rap1ide/slice_inflate/runs/hvvddqer",
      id=13,
      fold=2,
   ),

   ######## 14 ########
   "20240125__17_37_07_lazy-object_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/e975wi0p",
      id=14,
      fold=0,
   ),
   "20240125__17_37_07_wide-cube_stage-1_fold-1": dict(
      access_key="rap1ide/slice_inflate/runs/6plcdxdr",
      id=14,
      fold=1,
   ),
   "20240125__17_37_07_active-caramel_stage-1_fold-2": dict(
      access_key="rap1ide/slice_inflate/runs/w0bgwxn3",
      id=14,
      fold=2,
   ),

   ######## 15 ########
   "dummy45": dict(
      access_key=DUMMY_RUN,
      id=15,
      fold=0,
   ),
   "dummy46": dict(
      access_key=DUMMY_RUN,
      id=15,
      fold=1,
   ),
   "dummy47": dict(
      access_key=DUMMY_RUN,
      id=15,
      fold=2,
   ),
}


# Define run paths of wandb for MRXCAT

In [44]:
mrxcat_run_path_list = list(reversed([
    'rap1ide/slice_inflate/runs/egchj2m2',
    'rap1ide/slice_inflate/runs/4fe94wtl',
    'rap1ide/slice_inflate/runs/ihc5g4bg',
    'rap1ide/slice_inflate/runs/wnfu8s1l',
    'rap1ide/slice_inflate/runs/4j1oridg',
    'rap1ide/slice_inflate/runs/lc6itz1v',
    'rap1ide/slice_inflate/runs/t0jx7ix5',
    'rap1ide/slice_inflate/runs/qz5w3pcl',
    'rap1ide/slice_inflate/runs/g4xbtm4v',
    'rap1ide/slice_inflate/runs/ghja6j9o',
    'rap1ide/slice_inflate/runs/ofhiy0sq',
    'rap1ide/slice_inflate/runs/8w171q1e',
]))

In [45]:
mrxcat_run_name_list = list(reversed([
  '20240126__02_45_48_vibrant-mile_stage-3_fold-0',
  '20240125__22_24_43_staccato-specialist_stage-3_fold-0',
  '20240125__21_31_26_nervous-change_stage-4_fold-0',
  '20240125__18_43_41_lean-pinot_stage-1_fold-0',
  '20240125__18_03_05_each-practice_stage-1_fold-0',
  '20240125__17_21_39_approximate-seat_stage-1_fold-0',
  '20240125__16_46_51_refined-roman_stage-1_fold-0',
  '20240125__16_16_05_unproductive-trainer_stage-1_fold-0',
  '20240125__15_35_02_brass-cupola_stage-1_fold-0',
  '20240125__15_02_28_staccato-retail_stage-1_fold-0',
  '20240125__14_32_54_dry-berry_stage-1_fold-0',
  '20240125__14_15_08_grilled-shape_stage-1_fold-0',
]))

### Print run lists for convenience MRXCAT

In [46]:
for c_id, (nc, pc) in enumerate(zip(chunks(mrxcat_run_name_list, 1), chunks(mrxcat_run_path_list, 1))):
    print(f"## {c_id}")
    for n, p in zip(nc, pc):
        print(n, "\t", p)
print()

## 0
20240125__14_15_08_grilled-shape_stage-1_fold-0 	 rap1ide/slice_inflate/runs/8w171q1e
## 1
20240125__14_32_54_dry-berry_stage-1_fold-0 	 rap1ide/slice_inflate/runs/ofhiy0sq
## 2
20240125__15_02_28_staccato-retail_stage-1_fold-0 	 rap1ide/slice_inflate/runs/ghja6j9o
## 3
20240125__15_35_02_brass-cupola_stage-1_fold-0 	 rap1ide/slice_inflate/runs/g4xbtm4v
## 4
20240125__16_16_05_unproductive-trainer_stage-1_fold-0 	 rap1ide/slice_inflate/runs/qz5w3pcl
## 5
20240125__16_46_51_refined-roman_stage-1_fold-0 	 rap1ide/slice_inflate/runs/t0jx7ix5
## 6
20240125__17_21_39_approximate-seat_stage-1_fold-0 	 rap1ide/slice_inflate/runs/lc6itz1v
## 7
20240125__18_03_05_each-practice_stage-1_fold-0 	 rap1ide/slice_inflate/runs/4j1oridg
## 8
20240125__18_43_41_lean-pinot_stage-1_fold-0 	 rap1ide/slice_inflate/runs/wnfu8s1l
## 9
20240125__21_31_26_nervous-change_stage-4_fold-0 	 rap1ide/slice_inflate/runs/ihc5g4bg
## 10
20240125__22_24_43_staccato-specialist_stage-3_fold-0 	 rap1ide/slice_inflate/r

In [47]:
mrxcat_run_dict = {
   ######## 00 ########
   "20240125__14_15_08_grilled-shape_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/8w171q1e",
      id=0,
      fold=0,
   ),

   ######## 01 ########
   "20240125__14_32_54_dry-berry_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/ofhiy0sq",
      id=1,
      fold=0,
   ),

   ######## 02 ########
   "20240125__15_02_28_staccato-retail_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/ghja6j9o",
      id=2,
      fold=0,
   ),

   ######## 03 ########
   "20240125__15_35_02_brass-cupola_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/g4xbtm4v",
      id=3,
      fold=0,
   ),

   ######## 04 ########
   "20240125__16_16_05_unproductive-trainer_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/qz5w3pcl",
      id=4,
      fold=0,
   ),

   ######## 05 ########
   "20240125__16_46_51_refined-roman_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/t0jx7ix5",
      id=5,
      fold=0,
   ),

   ######## 06 ########
   "20240125__17_21_39_approximate-seat_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/lc6itz1v",
      id=6,
      fold=0,
   ),

   ######## 07 ########
   "20240125__18_03_05_each-practice_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/4j1oridg",
      id=7,
      fold=0,
   ),

   ######## 08 ########
   "20240125__18_43_41_lean-pinot_stage-1_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/wnfu8s1l",
      id=8,
      fold=0,
   ),

   ######## 09 ########
   "20240126__02_45_48_vibrant-mile_stage-3_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/egchj2m2",
      id=9,
      fold=0,
   ),

   ######## 10 ########
   "20240125__21_31_26_nervous-change_stage-4_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/ihc5g4bg",
      id=10,
      fold=0,
   ),

   ######## 11 ########
   "20240125__22_24_43_staccato-specialist_stage-3_fold-0": dict(
      access_key="rap1ide/slice_inflate/runs/4fe94wtl",
      id=11,
      fold=0,
   ),

   ######## 12 ########
   "20240126__15_59_14_navy-goat_stage-4_fold-0": dict(
      access_key="rap1ide/slice_inflate/etommcuz",
      id=12,
      fold=0,
   ),

   ######## 13 ########
   "dummy39": dict(
      access_key=DUMMY_RUN,
      id=13,
      fold=0,
   ),

   ######## 14 ########
   "20240126__16_26_42_future-stable_fold-0_stage-4": dict(
      access_key="rap1ide/slice_inflate/30v1u588",
      id=14,
      fold=0,
   ),

   ######## 15 ########
   "dummy45": dict(
      access_key=DUMMY_RUN,
      id=15,
      fold=0,
   ),
}

# Build latex tables

In [48]:
def get_wanted_keys(phase):
    assert phase in ['val', 'test']

    wanted_keys = {
        '_id': '_id',
        'prescan_res': 'Precsan props.',
        # 'description': 'Description',
        'prescan_type': 'Prescan props.',
        'slice_res': 'Slice resolution',
        'slice_type': 'Slice props.',
        'first_view': 'Slice view(s)',
        'second_view': 'Second view',
        'fold': 'fold',
        f'scores/{phase}_mean_dice_MYO': 'MYO',
        f'scores/{phase}_mean_dice_LV': 'LV',
        f'scores/{phase}_mean_dice_RV': 'RV',
        f'scores/{phase}_mean_dice_LA': 'LA',
        f'scores/{phase}_mean_dice_RA': 'RA',

        f'scores/{phase}_mean_oa_exclude_bg_dice': '\multicolumn{1}{c}{$\mu\pm\sigma$ }',
        f'scores/{phase}_std_oa_exclude_bg_dice': 'N/A',

        f'scores/{phase}_mean_hd95_MYO': 'MYO',
        f'scores/{phase}_mean_hd95_LV': 'LV',
        f'scores/{phase}_mean_hd95_RV': 'RV',
        f'scores/{phase}_mean_hd95_LA': 'LA',
        f'scores/{phase}_mean_hd95_RA': 'RA',
        f'scores/{phase}_mean_oa_exclude_bg_hd95': '\multicolumn{1}{c}{$\mu\pm\sigma$  }',
        f'scores/{phase}_std_oa_exclude_bg_hd95': 'N/A',

        # 'scores/val_mean_oa_exclude_bg_iou': 'IOU',
        # 'scores/val_std_oa_exclude_bg_iou': '',

        f'scores/{phase}_mean_delta_vol_rel_LV': '\multicolumn{1}{c}{$\mu\pm\sigma$}   ',
        f'scores/{phase}_std_delta_vol_rel_LV': '',
    }
    return wanted_keys

# Build latex table for MMWHS

In [49]:
df = pd.DataFrame()

for run_idx, (wandb_run_name, rr) in enumerate(mmwhs_run_dict.items()):
   run_key = rr['access_key']
   run_numeric_id = rr['id']
   try:
      wrun = api.run(run_key)
   except:
      print(f"Failed to fetch run {run_key}")
      continue

   run_frame = pd.DataFrame(wrun.history())
   run_frame = run_frame.iloc[-1:]
   run_frame.index = [run_idx]

   if 'dummy' in wandb_run_name:
      run_frame = run_frame.map(lambda x: 0)

   if 3 <= run_numeric_id < 9:
      run_settings = settings['3:9']
   else:
      run_settings = settings[f"{run_numeric_id}:{run_numeric_id+1}"]

   run_frame.insert(0, '_id', [rr['id']])
   run_frame.insert(1, 'description', [run_settings['description']])
   run_frame.insert(2, 'first_view', run_settings['first_view'])
   run_frame.insert(3, 'second_view', run_settings['second_view'])
   run_frame.insert(4, 'prescan_res', run_settings['prescan_res'])
   run_frame.insert(5, 'prescan_type', run_settings['prescan_type'])
   run_frame.insert(6, 'slice_res', run_settings['slice_res'])
   run_frame.insert(7, 'slice_type', run_settings['slice_type'])
   run_frame.insert(8, 'fold', rr['fold'])

   df = pd.concat([df,run_frame])

In [52]:
filtered_frame = df.copy()
# filtered_frame = filtered_frame.groupby('_id').agg(get_agg_dict(filtered_frame)) # TODO readd
filtered_frame = filtered_frame[filtered_frame['fold'] == 0]

# Prepare values
wanted_keys = get_wanted_keys('val')
filtered_frame = filtered_frame[wanted_keys.keys()]

In [53]:
percent_keys = [k for k in wanted_keys if "dice" in k or "vol_rel" in k]
mean_keys = [k for k in wanted_keys if "mean" in k]
min_metrics_keys = [k for k in wanted_keys if ("delta" in k or "hd" in k) and not 'std' in k]
max_metrics_keys = [k for k in wanted_keys if ("dice" in k or "iou" in k) and not 'std' in k]
filtered_frame.loc[:,percent_keys] *= 100.

# Aggregate random runs to build mean
random_runs = list(range(0,3)) + 6 * ['is_random'] + list(range(9,16))
filtered_frame.insert(1, 'is_random', random_runs)
filtered_frame = filtered_frame.groupby('is_random', as_index=False).agg(get_agg_dict(filtered_frame))

reindex_idx = list(range(0,3)) + [-1] + list(range(3,10))
filtered_frame = filtered_frame.iloc[reindex_idx,:]
filtered_frame.drop('is_random', axis=1, inplace=True)

filtered_frame

,_id,prescan_res,prescan_type,slice_res,slice_type,first_view,second_view,fold,scores/val_mean_dice_MYO,scores/val_mean_dice_LV,...,scores/val_std_oa_exclude_bg_dice,scores/val_mean_hd95_MYO,scores/val_mean_hd95_LV,scores/val_mean_hd95_RV,scores/val_mean_hd95_LA,scores/val_mean_hd95_RA,scores/val_mean_oa_exclude_bg_hd95,scores/val_std_oa_exclude_bg_hd95,scores/val_mean_delta_vol_rel_LV,scores/val_std_delta_vol_rel_LV
0,0,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,p2CH,p4CH,0,82.142175,90.306582,...,9.106091,6.870135,6.810167,17.015399,17.216680,21.404289,13.863334,8.802993,10.073156,12.400881
1,1,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,2CH,4CH,0,76.794281,86.147047,...,10.533144,8.182123,9.016636,18.421243,24.621676,40.586437,20.165623,22.571114,21.294219,28.829897
2,2,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,2CH,SA,0,79.800909,87.195365,...,11.056108,7.923197,11.253205,16.493283,26.129946,32.971625,18.954251,10.893020,14.684358,11.128303
10,3,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,RND,RND,0,70.939009,82.228079,...,9.947269,11.942087,13.756238,20.057258,20.189303,22.166498,17.622277,7.929344,21.275417,22.692167
3,9,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,OPT,OPT,0,77.502906,87.132563,...,8.212827,8.987140,9.577253,17.676619,15.590535,18.904630,14.147235,8.434314,17.847576,17.500742
4,10,$(6mm)^3$,GT,$(1.5mm)^2$,GT,2CH,4CH,0,77.057711,86.644510,...,10.304800,8.179473,12.881558,15.361105,12.439336,22.202379,14.212770,7.094578,16.061512,22.426183
5,11,$(6mm)^3$,GT,$(1.5mm)^2$,GT,OPT,OPT,0,61.893681,79.442217,...,11.345707,15.681330,16.696080,23.534056,24.976980,26.877014,21.553092,7.889524,22.934789,24.887887
6,12,$(6mm)^3$,SEG,$(1.5mm)^2$,SEG,2CH,4CH,0,62.534293,76.398351,...,16.669546,17.574653,21.322394,26.790203,54.030858,46.546808,33.252983,23.329723,24.240410,19.291840
7,13,$(6mm)^3$,SEG,$(1.5mm)^2$,SEG,OPT,OPT,0,65.136047,79.902714,...,21.272924,12.338927,15.630103,29.067765,31.980374,40.161248,25.835683,24.386463,15.008021,6.537549
8,14,$(6mm)^3$,SEG,$(6mm)^3$,SEG,2CH,4CH,0,55.953734,75.518045,...,17.718837,19.613987,20.584038,28.734546,31.333682,34.764329,27.006117,10.403451,16.092807,15.381128


In [54]:
# Get bold values (best)
group_ranges = [
    [0,5],
    [5,7],
    [7,9],
    [9,11],
]

bold_idxs = []
for sub_range in group_ranges:
    sub_frame = filtered_frame.iloc[sub_range[0]:sub_range[1]]
    optimal_vals_maximize = sub_frame[max_metrics_keys].idxmax()
    optimal_vals_minimize = sub_frame[min_metrics_keys].idxmin()

    bold_idxs.append(optimal_vals_maximize)
    bold_idxs.append(optimal_vals_minimize)

In [55]:
# Round and convert
filtered_frame = filtered_frame.round(decimals=1)
string_frame = filtered_frame.copy().astype(str)

# Fuse mean and std columns
for m in mean_keys:
    mean_col = string_frame[m].astype(str)
    std_col_key = m.replace("mean","std")
    if std_col_key in string_frame.columns:
        std_col = string_frame[std_col_key].astype(str)
        string_frame[m] = mean_col + r" \pm " + std_col
        string_frame.drop(std_col_key, axis=1, inplace=True)

# Add bold to optimal values
for bold_group in bold_idxs:
    for col_name, idx in zip(bold_group.index, bold_group.values):
        row_idx = string_frame.index.get_loc(idx)
        col_idx = string_frame.columns.get_loc(col_name)
        string_frame.iloc[row_idx,col_idx] = r"\B " + string_frame.iloc[row_idx,col_idx]

# Join first and second view
string_frame = join_cols(string_frame, 'first_view', 'second_view', join_str=r"+")
string_frame = join_cols(string_frame, 'prescan_type', 'prescan_res', join_str=r" ", drop_second=True)
string_frame = join_cols(string_frame, 'slice_type', 'slice_res', join_str=r" ", drop_second=True)

# Replace column names
string_frame.columns = [wanted_keys[c] for c in string_frame.columns]

# Drop and insert spacer columns
string_frame.drop('_id', axis=1, inplace=True)
string_frame.drop('fold', axis=1, inplace=True)

string_frame.insert(3, ' ', len(string_frame)*["\hspace{1pt}"])
string_frame.insert(10, '  ', len(string_frame)*["\hspace{1pt}"])
string_frame.insert(17, '   ', len(string_frame)*["\hspace{1pt}"])

# Add offset to groups in latex
for group_idxs in group_ranges[:-1]:
    txt = string_frame.iloc[group_idxs[1],0]
    string_frame.iloc[group_idxs[1],0] = r"\rule{0pt}{4ex} "+txt

string_frame

,Prescan props.,Slice props.,Slice view(s),,MYO,LV,RV,LA,RA,\multicolumn{1}{c}{$\mu\pm\sigma$ },,MYO,LV,RV,LA,RA,\multicolumn{1}{c}{$\mu\pm\sigma$ },,\multicolumn{1}{c}{$\mu\pm\sigma$}
0,GT $(1.5mm)^3$,GT $(1.5mm)^2$,p2CH+p4CH,\hspace{1pt},\B 82.1,\B 90.3,77.6,79.3,73.8,\B 80.6 \pm 9.1,\hspace{1pt},\B 6.9,\B 6.8,17.0,17.2,21.4,\B 13.9 \pm 8.8,\hspace{1pt},\B 10.1 \pm 12.4
1,GT $(1.5mm)^3$,GT $(1.5mm)^2$,2CH+4CH,\hspace{1pt},76.8,86.1,76.7,\B 80.5,68.7,77.8 \pm 10.5,\hspace{1pt},8.2,9.0,18.4,24.6,40.6,20.2 \pm 22.6,\hspace{1pt},21.3 \pm 28.8
2,GT $(1.5mm)^3$,GT $(1.5mm)^2$,2CH+SA,\hspace{1pt},79.8,87.2,77.6,69.1,60.9,74.9 \pm 11.1,\hspace{1pt},7.9,11.3,\B 16.5,26.1,33.0,19.0 \pm 10.9,\hspace{1pt},14.7 \pm 11.1
10,GT $(1.5mm)^3$,GT $(1.5mm)^2$,RND+RND,\hspace{1pt},70.9,82.2,76.2,71.4,70.0,74.2 \pm 9.9,\hspace{1pt},11.9,13.8,20.1,20.2,22.2,17.6 \pm 7.9,\hspace{1pt},21.3 \pm 22.7
3,GT $(1.5mm)^3$,GT $(1.5mm)^2$,OPT+OPT,\hspace{1pt},77.5,87.1,\B 81.0,79.7,\B 77.0,80.5 \pm 8.2,\hspace{1pt},9.0,9.6,17.7,\B 15.6,\B 18.9,14.1 \pm 8.4,\hspace{1pt},17.8 \pm 17.5
4,\rule{0pt}{4ex} GT $(6mm)^3$,GT $(1.5mm)^2$,2CH+4CH,\hspace{1pt},\B 77.1,\B 86.6,\B 78.5,\B 83.5,\B 70.8,\B 79.3 \pm 10.3,\hspace{1pt},\B 8.2,\B 12.9,\B 15.4,\B 12.4,\B 22.2,\B 14.2 \pm 7.1,\hspace{1pt},\B 16.1 \pm 22.4
5,GT $(6mm)^3$,GT $(1.5mm)^2$,OPT+OPT,\hspace{1pt},61.9,79.4,67.4,65.6,68.3,68.5 \pm 11.3,\hspace{1pt},15.7,16.7,23.5,25.0,26.9,21.6 \pm 7.9,\hspace{1pt},22.9 \pm 24.9
6,\rule{0pt}{4ex} SEG $(6mm)^3$,SEG $(1.5mm)^2$,2CH+4CH,\hspace{1pt},62.5,76.4,70.3,46.3,48.9,60.9 \pm 16.7,\hspace{1pt},17.6,21.3,\B 26.8,54.0,46.5,33.3 \pm 23.3,\hspace{1pt},24.2 \pm 19.3
7,SEG $(6mm)^3$,SEG $(1.5mm)^2$,OPT+OPT,\hspace{1pt},\B 65.1,\B 79.9,\B 71.6,\B 56.2,\B 58.6,\B 66.3 \pm 21.3,\hspace{1pt},\B 12.3,\B 15.6,29.1,\B 32.0,\B 40.2,\B 25.8 \pm 24.4,\hspace{1pt},\B 15.0 \pm 6.5
8,\rule{0pt}{4ex} SEG $(6mm)^3$,SEG $(6mm)^3$,2CH+4CH,\hspace{1pt},\B 56.0,\B 75.5,\B 61.4,\B 52.7,\B 48.8,\B 58.9 \pm 17.7,\hspace{1pt},19.6,20.6,28.7,31.3,34.8,27.0 \pm 10.4,\hspace{1pt},16.1 \pm 15.4


In [18]:
# Save to latex

PM_COL_FORMAT = "S[table-figures-decimal=1,separate-uncertainty=true,table-format=3.1(3)]"
COL_FORMAT = \
    ("c" * 9) \
    + PM_COL_FORMAT \
    + ("c" * 6) \
    + PM_COL_FORMAT \
    + "c" \
    + PM_COL_FORMAT
# 19 cols

header = [r'\multicolumn{3}{c}{\textbf{Experiment I}}'] \
    + 2*[None] \
    + [r'\hspace{1pt}'] \
    + [r'\multicolumn{6}{c}{\textbf{Dice in \% $\uparrow$}}'] + 5*[None] \
    + [r'\hspace{1pt}'] \
    + [r'\multicolumn{6}{c}{\textbf{HD95 in mm $\downarrow$}}'] + 5*[None] \
    + [r'\hspace{1pt}'] \
    + [r'\textbf{$\Delta$vol LV in \% $\downarrow$}']

latex_frame = pd.concat([pd.DataFrame(header, index=string_frame.columns).T, string_frame])

latex_frame.to_latex(
    buf="mmwhs_results.txt",
    escape=False,
    column_format=COL_FORMAT,
    index=False,
)

# Load latex file and replace & NaN with &
with open("mmwhs_results.txt", "r") as f:
    lines = f.readlines()

    lines = [l.replace("& NaN", "") for l in lines]

with open("mmwhs_results.txt", "w") as f:
    f.writelines(lines)

# Insert 5th line before third line
with open("mmwhs_results.txt", "r") as f:
    lines = f.readlines()
    lines.insert(2, lines[4])
    del lines[5]
with open("mmwhs_results.txt", "w") as f:
    f.writelines(lines)

# Build latex table for MRXCAT

In [19]:
df = pd.DataFrame()

for run_idx, (wandb_run_name, rr) in enumerate(mrxcat_run_dict.items()):
   run_key = rr['access_key']
   run_numeric_id = rr['id']
   try:
      wrun = api.run(run_key)
   except:
      print(f"Failed to fetch run {run_key}")
      continue

   run_frame = pd.DataFrame(wrun.history())
   run_frame = run_frame.iloc[-1:]
   run_frame.index = [run_idx]

   if 'dummy' in wandb_run_name:
      run_frame = run_frame.map(lambda x: 0)

   if 3 <= run_numeric_id < 9:
      run_settings = settings['3:9']
   else:
      run_settings = settings[f"{run_numeric_id}:{run_numeric_id+1}"]

   run_frame.insert(0, '_id', [rr['id']])
   run_frame.insert(1, 'description', [run_settings['description']])
   run_frame.insert(2, 'first_view', run_settings['first_view'])
   run_frame.insert(3, 'second_view', run_settings['second_view'])
   run_frame.insert(4, 'prescan_res', run_settings['prescan_res'])
   run_frame.insert(5, 'prescan_type', run_settings['prescan_type'])
   run_frame.insert(6, 'slice_res', run_settings['slice_res'])
   run_frame.insert(7, 'slice_type', run_settings['slice_type'])
   run_frame.insert(8, 'fold', rr['fold'])

   df = pd.concat([df,run_frame])

df

,_id,description,first_view,second_view,prescan_res,prescan_type,slice_res,slice_type,fold,scores/train_std_delta_vol_ml_RV,...,orientations/test_sa_theta_t_offsets0_mean,orientations/test_sa_theta_ap2_std,orientations/train_hla_theta_ap4_std,orientations/test_hla_theta_ap1_mean,orientations/val_hla_theta_zp0_mean,orientations/test_sa_theta_ap0_std,orientations/test_sa_theta_ap1_mean,orientations/val_sa_theta_zp0_std,orientations/train_hla_theta_ap3_std,orientations/train_hla_theta_t_offsets2_mean
0,0,Clinical standard,p2CH,p4CH,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,0,1.636645,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Clinical standard,2CH,4CH,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,0,2.165242,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Clinical standard,2CH,SA,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,0,2.698917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Mean out of 6 Random,RND,RND,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,0,4.070517,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Mean out of 6 Random,RND,RND,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,0,2.535359,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Mean out of 6 Random,RND,RND,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,0,3.828898,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Mean out of 6 Random,RND,RND,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,0,3.582119,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Mean out of 6 Random,RND,RND,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,0,3.381246,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Mean out of 6 Random,RND,RND,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,0,2.665526,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Optimized,OPT,OPT,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,0,14.252794,...,-0.023438,0.007351,0.447280,14.395677,1.0,0.023855,2.723526,0.0,0.617823,-0.070313


In [20]:
filtered_frame = df.copy()
filtered_frame = filtered_frame.groupby('_id').agg(get_agg_dict(filtered_frame))

wanted_keys = get_wanted_keys('test') # Use test here for MRXCAT

In [21]:
# Prepare values
filtered_frame = filtered_frame[wanted_keys.keys()]

percent_keys = [k for k in wanted_keys if "dice" in k or "vol_rel" in k]
mean_keys = [k for k in wanted_keys if "mean" in k]
min_metrics_keys = [k for k in wanted_keys if ("delta" in k or "hd" in k) and not 'std' in k]
max_metrics_keys = [k for k in wanted_keys if ("dice" in k or "iou" in k) and not 'std' in k]
filtered_frame.loc[:,percent_keys] *= 100.

# Aggregate random runs to build mean
random_runs = list(range(0,3)) + 6 * ['is_random'] + list(range(9,16))
filtered_frame.insert(1, 'is_random', random_runs)
filtered_frame = filtered_frame.groupby('is_random', as_index=False).agg(get_agg_dict(filtered_frame))

reindex_idx = list(range(0,3)) + [-1] + list(range(3,10))
filtered_frame = filtered_frame.iloc[reindex_idx,:]
filtered_frame.drop('is_random', axis=1, inplace=True)

filtered_frame

,_id,prescan_res,prescan_type,slice_res,slice_type,first_view,second_view,fold,scores/test_mean_dice_MYO,scores/test_mean_dice_LV,...,scores/test_std_oa_exclude_bg_dice,scores/test_mean_hd95_MYO,scores/test_mean_hd95_LV,scores/test_mean_hd95_RV,scores/test_mean_hd95_LA,scores/test_mean_hd95_RA,scores/test_mean_oa_exclude_bg_hd95,scores/test_std_oa_exclude_bg_hd95,scores/test_mean_delta_vol_rel_LV,scores/test_std_delta_vol_rel_LV
0,0,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,p2CH,p4CH,0,84.641796,89.905107,...,3.841510,5.006896,5.481754,6.514866,7.524068,7.246971,6.354911,1.654760,21.401555,13.816542
1,1,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,2CH,4CH,0,76.878982,85.586771,...,3.961466,6.094784,6.461951,7.552638,7.859861,9.500514,7.493949,1.785818,33.920090,13.669775
2,2,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,2CH,SA,0,82.850911,89.889262,...,6.484449,5.579479,5.645747,15.448795,11.598800,13.667210,10.388006,5.381621,21.311510,9.468760
10,3,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,RND,RND,0,80.936613,87.501125,...,5.286140,7.515442,7.796290,10.717961,10.048375,11.308519,9.477318,3.516369,23.495407,12.508354
3,9,$(1.5mm)^3$,GT,$(1.5mm)^2$,GT,OPT,OPT,0,74.853980,86.274450,...,5.826232,7.899631,7.062600,5.884158,12.272633,9.148672,8.453539,3.088667,29.449003,16.144714
4,10,$(6mm)^3$,GT,$(1.5mm)^2$,GT,2CH,4CH,0,85.769944,90.765040,...,4.281002,4.898186,14.383776,6.328859,9.484782,10.113405,9.041802,8.460434,17.118691,10.181439
5,11,$(6mm)^3$,GT,$(1.5mm)^2$,GT,OPT,OPT,0,75.110361,83.801749,...,6.160986,7.205254,7.734472,5.700679,8.152070,9.602473,7.678990,2.252497,39.924002,21.947657
6,12,$(6mm)^3$,SEG,$(1.5mm)^2$,SEG,2CH,4CH,0,70.274425,81.067562,...,6.262354,7.627830,8.427763,8.130813,8.532231,10.996769,8.743081,2.596051,48.438331,21.510787
7,13,$(6mm)^3$,SEG,$(1.5mm)^2$,SEG,OPT,OPT,0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,14,$(6mm)^3$,SEG,$(6mm)^3$,SEG,2CH,4CH,0,79.952453,89.927898,...,4.030221,6.017407,5.274241,6.658468,7.465805,8.546093,6.792403,1.569478,18.916123,6.733484


In [22]:
# Get bold values (best)
group_ranges = [
    [0,5],
    [5,7],
    [7,9],
    [9,11],
]

bold_idxs = []
for sub_range in group_ranges:
    sub_frame = filtered_frame.iloc[sub_range[0]:sub_range[1]]
    optimal_vals_maximize = sub_frame[max_metrics_keys].idxmax()
    optimal_vals_minimize = sub_frame[min_metrics_keys].idxmin()

    bold_idxs.append(optimal_vals_maximize)
    bold_idxs.append(optimal_vals_minimize)

In [23]:
# Round and convert
filtered_frame = filtered_frame.round(decimals=1)
string_frame = filtered_frame.copy().astype(str)

# Fuse mean and std columns
for m in mean_keys:
    mean_col = string_frame[m].astype(str)
    std_col_key = m.replace("mean","std")
    if std_col_key in string_frame.columns:
        std_col = string_frame[std_col_key].astype(str)
        string_frame[m] = mean_col + r" \pm " + std_col
        string_frame.drop(std_col_key, axis=1, inplace=True)

# Add bold to optimal values
for bold_group in bold_idxs:
    for col_name, idx in zip(bold_group.index, bold_group.values):
        row_idx = string_frame.index.get_loc(idx)
        col_idx = string_frame.columns.get_loc(col_name)
        string_frame.iloc[row_idx,col_idx] = r"\B " + string_frame.iloc[row_idx,col_idx]

# Join first and second view
string_frame = join_cols(string_frame, 'first_view', 'second_view', join_str=r"+")
string_frame = join_cols(string_frame, 'prescan_type', 'prescan_res', join_str=r" ", drop_second=True)
string_frame = join_cols(string_frame, 'slice_type', 'slice_res', join_str=r" ", drop_second=True)

# Replace column names
string_frame.columns = [wanted_keys[c] for c in string_frame.columns]

# Drop and insert spacer columns
string_frame.drop('_id', axis=1, inplace=True)
string_frame.drop('fold', axis=1, inplace=True)

string_frame.insert(3, ' ', len(string_frame)*["\hspace{1pt}"])
string_frame.insert(10, '  ', len(string_frame)*["\hspace{1pt}"])
string_frame.insert(17, '   ', len(string_frame)*["\hspace{1pt}"])

# Add offset to groups in latex
for group_idxs in group_ranges[:-1]:
    txt = string_frame.iloc[group_idxs[1],0]
    string_frame.iloc[group_idxs[1],0] = r"\rule{0pt}{4ex} "+txt

string_frame

,Prescan props.,Slice props.,Slice view(s),,MYO,LV,RV,LA,RA,\multicolumn{1}{c}{$\mu\pm\sigma$ },,MYO,LV,RV,LA,RA,\multicolumn{1}{c}{$\mu\pm\sigma$ },,\multicolumn{1}{c}{$\mu\pm\sigma$}
0,GT $(1.5mm)^3$,GT $(1.5mm)^2$,p2CH+p4CH,\hspace{1pt},\B 84.6,\B 89.9,87.3,\B 83.9,\B 85.6,\B 86.3 \pm 3.8,\hspace{1pt},\B 5.0,\B 5.5,6.5,\B 7.5,\B 7.2,\B 6.4 \pm 1.7,\hspace{1pt},21.4 \pm 13.8
1,GT $(1.5mm)^3$,GT $(1.5mm)^2$,2CH+4CH,\hspace{1pt},76.9,85.6,84.2,81.6,82.9,82.2 \pm 4.0,\hspace{1pt},6.1,6.5,7.6,7.9,9.5,7.5 \pm 1.8,\hspace{1pt},33.9 \pm 13.7
2,GT $(1.5mm)^3$,GT $(1.5mm)^2$,2CH+SA,\hspace{1pt},82.9,89.9,78.6,79.4,73.2,80.8 \pm 6.5,\hspace{1pt},5.6,5.6,15.4,11.6,13.7,10.4 \pm 5.4,\hspace{1pt},\B 21.3 \pm 9.5
10,GT $(1.5mm)^3$,GT $(1.5mm)^2$,RND+RND,\hspace{1pt},80.9,87.5,84.5,80.3,79.3,82.5 \pm 5.3,\hspace{1pt},7.5,7.8,10.7,10.0,11.3,9.5 \pm 3.5,\hspace{1pt},23.5 \pm 12.5
3,GT $(1.5mm)^3$,GT $(1.5mm)^2$,OPT+OPT,\hspace{1pt},74.9,86.3,\B 91.0,82.3,83.9,83.7 \pm 5.8,\hspace{1pt},7.9,7.1,\B 5.9,12.3,9.1,8.5 \pm 3.1,\hspace{1pt},29.4 \pm 16.1
4,\rule{0pt}{4ex} GT $(6mm)^3$,GT $(1.5mm)^2$,2CH+4CH,\hspace{1pt},\B 85.8,\B 90.8,88.7,83.5,80.8,\B 85.9 \pm 4.3,\hspace{1pt},\B 4.9,14.4,6.3,9.5,10.1,9.0 \pm 8.5,\hspace{1pt},\B 17.1 \pm 10.2
5,GT $(6mm)^3$,GT $(1.5mm)^2$,OPT+OPT,\hspace{1pt},75.1,83.8,\B 89.4,\B 86.0,\B 82.9,83.4 \pm 6.2,\hspace{1pt},7.2,\B 7.7,\B 5.7,\B 8.2,\B 9.6,\B 7.7 \pm 2.3,\hspace{1pt},39.9 \pm 21.9
6,\rule{0pt}{4ex} SEG $(6mm)^3$,SEG $(1.5mm)^2$,2CH+4CH,\hspace{1pt},\B 70.3,\B 81.1,\B 84.7,\B 81.9,\B 81.9,\B 80.0 \pm 6.3,\hspace{1pt},7.6,8.4,8.1,8.5,11.0,8.7 \pm 2.6,\hspace{1pt},48.4 \pm 21.5
7,SEG $(6mm)^3$,SEG $(1.5mm)^2$,OPT+OPT,\hspace{1pt},0.0,0.0,0.0,0.0,0.0,0.0 \pm 0.0,\hspace{1pt},\B 0.0,\B 0.0,\B 0.0,\B 0.0,\B 0.0,\B 0.0 \pm 0.0,\hspace{1pt},\B 0.0 \pm 0.0
8,\rule{0pt}{4ex} SEG $(6mm)^3$,SEG $(6mm)^3$,2CH+4CH,\hspace{1pt},\B 80.0,\B 89.9,\B 86.8,\B 81.5,\B 84.0,\B 84.5 \pm 4.0,\hspace{1pt},6.0,5.3,6.7,7.5,8.5,6.8 \pm 1.6,\hspace{1pt},18.9 \pm 6.7


In [24]:
# Save to latex

PM_COL_FORMAT = "S[table-figures-decimal=1,separate-uncertainty=true,table-format=3.1(3)]"
COL_FORMAT = \
    ("c" * 9) \
    + PM_COL_FORMAT \
    + ("c" * 6) \
    + PM_COL_FORMAT \
    + "c" \
    + PM_COL_FORMAT
# 19 cols

header = [r'\multicolumn{3}{c}{\textbf{Experiment II}}'] \
    + 2*[None] \
    + [r'\hspace{1pt}'] \
    + [r'\multicolumn{6}{c}{\textbf{Dice in \% $\uparrow$}}'] + 5*[None] \
    + [r'\hspace{1pt}'] \
    + [r'\multicolumn{6}{c}{\textbf{HD95 in mm $\downarrow$}}'] + 5*[None] \
    + [r'\hspace{1pt}'] \
    + [r'\textbf{$\Delta$vol LV in \% $\downarrow$}']

latex_frame = pd.concat([pd.DataFrame(header, index=string_frame.columns).T, string_frame])

latex_frame.to_latex(
    buf="mrxcat_results.txt",
    escape=False,
    column_format=COL_FORMAT,
    index=False,
)

# Load latex file and replace & NaN with &
with open("mrxcat_results.txt", "r") as f:
    lines = f.readlines()

    lines = [l.replace("& NaN", "") for l in lines]

with open("mrxcat_results.txt", "w") as f:
    f.writelines(lines)

# Insert 5th line before third line
with open("mrxcat_results.txt", "r") as f:
    lines = f.readlines()
    lines.insert(2, lines[4])
    del lines[5]
with open("mrxcat_results.txt", "w") as f:
    f.writelines(lines)